In [1]:
%load_ext autoreload
%autoreload 2
%load_ext Cython
%reload_ext Cython

In [2]:
import pandas as pd
import numpy as np

import pyarrow.parquet as pq

import time
import datetime as dt


from pathlib import Path

#### Introduction: Matching with Outcomes from Biometric Data
Main Challenges:
1. <b>Define Starting and Ending Dates of the Study (Canary Registration and End Date Could be an indication)
2. <b> After Defining the Dates, find the corresponding Measurements (There might be no data in the exact dates) </b><br>
    Solutions:
    1. Take the dates closer to registration and end dates (up to 6 months), else remove the patient from study (not enough data)
    2. Average the covariates in a period of at most 6 months from the start and end date (as well as the outcomes)
3. <b> Do the same for the control group (idealy the control group should come from the same distribution in time), this is due to the problem
of covariate shift (the means of the measurements of key features change over time, data are registered in different ways, different standards etc)
    
Include Continuous Enrollment Criteria

#### LOAD DATA:
1. <b>CANARY(TREATED)
2. <b>BIOMETRICS(OUTCOME)
3. <b>PMPM(ELIGIBLE)

In [3]:
%%time
canary_p = '../../data/CANARY/CANARY/PERSON_SUMMARY'
biometrics_p = '../../data/BIOMETRICS'
pmpm_p = '../../temp/PMPM.p'

canary = pq.read_table(canary_p).to_pandas()
biometrics = pq.read_table(biometrics_p).to_pandas()
pmpm = pd.read_pickle(pmpm_p)


CPU times: user 33.9 s, sys: 10.3 s, total: 44.1 s
Wall time: 25.5 s


In [4]:
%%time
print(type(biometrics.RESULT_VALUE[0]))
biometrics.RESULT_VALUE = biometrics.RESULT_VALUE.astype(float)

<class 'str'>
CPU times: user 14.8 s, sys: 2.49 s, total: 17.3 s
Wall time: 17.3 s


#### Length of Datasets (# Rows)

In [5]:
print("CANARY N:{}\nBIO N:{}\nPMPM N:{}".format(len(canary), len(biometrics), len(pmpm)))

CANARY N:1237
BIO N:58209250
PMPM N:3017136


#### Biometric Categories

In [6]:
%%time
categories = biometrics.groupby(by = 'BIOMETRIC_CATEGORY')['BIOMETRIC_CATEGORY'].apply(len)
print(categories)

BIOMETRIC_CATEGORY
A1C                             740341
BMI                            9607665
BNP                              39357
Blood Pressure (Diastolic)     4640018
Blood Pressure (Systolic)      4934403
Cholesterol (LDL)              1240007
DCSI-EGFR                       592754
DCSI-MICROALBUMIN                77517
DCSI-SERUM                     2576985
Ferritin                        207220
Height                        11179070
Hematocrit                     2401459
Hemoglobin                     2735398
Iron                            185396
NT_proBNP                         8261
PSA                             299144
Potassium                      2490129
Troponin                         87111
Weight                        13305577
eGFR                            861438
Name: BIOMETRIC_CATEGORY, dtype: int64
CPU times: user 6.57 s, sys: 2.09 s, total: 8.66 s
Wall time: 8.66 s


#### Pick Category as Outcome
1. <b>BMI
2. <b>A1C
3. <b>Blood Pressure (Diatolic)
4. <b>Blood Pressure (Systolic)
5. <b>Cholesterol (LDL)
6. <b>Other

In [7]:
%%time
cat = 'BMI'
filt = biometrics['BIOMETRIC_CATEGORY'] == cat
bio_cat = biometrics[filt]

CPU times: user 3.64 s, sys: 344 ms, total: 3.98 s
Wall time: 3.97 s


#### Find Common Ids from:
1. Canary
2. Biometics
3. PMPM 

We need common ids between PMPM and Biometrics and common ids between Canary and Biometrics

In [8]:
pmpm_ids = pmpm.PERS_ID.astype(int).unique()
canary_ids = canary.PERS_ID.astype(int).unique()
biometrics_ids = bio_cat.PERS_ID.astype(int).unique()

In [9]:
common_ids = np.intersect1d(pmpm_ids, biometrics_ids)
common_ids2 = np.intersect1d(common_ids, canary_ids)

In [10]:
len(common_ids), len(common_ids2)

(116736, 1122)

#### Filter data to contain only common ids

In [11]:
%%time
filter1 = pmpm.PERS_ID.astype(int).isin(common_ids)
filter2 = bio_cat.PERS_ID.astype(int).isin(common_ids)
filter3 = canary.PERS_ID.astype(int).isin(common_ids)

pmpmf = pmpm[filter1]
bio_catf = bio_cat[filter2]
canaryf = canary[filter3]

CPU times: user 3.74 s, sys: 1.94 s, total: 5.68 s
Wall time: 5.68 s


#### Turn IDs to Integers

In [12]:
pmpmf['PERS_ID'] = pmpmf['PERS_ID'].astype(int)
bio_catf['PERS_ID'] = bio_catf['PERS_ID'].astype(int)
canaryf['PERS_ID'] = canaryf['PERS_ID'].astype(int)

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [13]:
len(canaryf), len(bio_catf), len(pmpmf)

(1122, 5002343, 2558652)

#### Standarize the dates into Year Month Format

In [14]:
def MYR_to_DATE(row):
    """
    transform MYR to datetime.date
    
    """
    year = row//100
    month = row%100
    
    return dt.date(year = year, month = month, day = 1)

In [15]:
pmpmf['MYR_DATE'] = pmpmf.MYR.apply(MYR_to_DATE)
bio_catf['EVENT_DATE2'] = pd.to_datetime(bio_catf.EVENT_DATE).dt.date

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [16]:
bio_catf.head()

PERS_ID           EVENT_DATE  DATE_TYPE BIOMETRIC_CATEGORY EVENT_CODE  \
0  2570719  2019-10-02 20:22:36  CALC_DATE                BMI       None   
4  2570719  2018-12-04 19:34:56  CALC_DATE                BMI       None   
5  2570719  2005-11-08 08:30:00  CALC_DATE                BMI       None   
8  2570719  2018-12-07 09:33:28  CALC_DATE                BMI       None   
9  2570719  2019-12-18 19:00:00  CALC_DATE                BMI       None   

  CODING_STANDARD      SOURCE  RESULT_VALUE EVENT_DATE2  
0            None  CALCULATED     25.537492  2019-10-02  
4            None  CALCULATED     26.398623  2018-12-04  
5            None  CALCULATED     28.671601  2005-11-08  
8            None  CALCULATED     26.398623  2018-12-07  
9            None  CALCULATED     25.250612  2019-12-18

In [17]:
canaryf.head()

PERS_ID       MBR_ID GROUPNUMBER               GROUPNAME       LOB  \
0  2638452  M0006106300       37330  Medicare Advantage 175  Medicare   
1  2706146  M0010332600       36550  Medicare Advantage 125  Medicare   
2  4122088  CD116486701       35938          365 LifePoints        LG   
3  4254340  CD127136501       35938          365 LifePoints        LG   
5  2884594  3H857647700       36267          180 LifePoints        LG   

                                           FILE_NAME   FILE_DATE  \
0  s3://cdphp-s3-us-e-p-datalake/RAW/NATIVE/WELLT...  2020-09-07   
1  s3://cdphp-s3-us-e-p-datalake/RAW/NATIVE/WELLT...  2020-09-07   
2  s3://cdphp-s3-us-e-p-datalake/RAW/NATIVE/WELLT...  2020-09-07   
3  s3://cdphp-s3-us-e-p-datalake/RAW/NATIVE/WELLT...  2020-09-07   
5  s3://cdphp-s3-us-e-p-datalake/RAW/NATIVE/WELLT...  2020-09-07   

   N_MILESTONES_ACHIEVED REGISTER_DATE LAST_ACTIVITY_DATE  \
0                      1    2020-01-26         2020-01-26   
1                      9    2020-05-27         2020-08-31   
2                     17    2019-11-20         2020-05-29   
3                     17    2019-02-04         2019-07-24   
5                      9    2020-06-10         2020-09-04   

    LAST_COMPLETED_MILESTONE  
0              REGISTER_DATE  
1   9TH_LESSON_COMPLETE_DATE  
2  17TH_LESSON_COMPLETE_DATE  
3  17TH_LESSON_COMPLETE_DATE  
5   9TH_LESSON_COMPLETE_DATE

In [18]:
pmpmf.head()

PERS_ID  AGE_AT_MIDMONTH  ALLOW_AMT  CC_ADHD  CC_ALZHEIMER  \
0  2570268               94        196        0             1   
1  2570268               95         95        0             1   
2  2570268               96        241        0             0   
3  2570268               94        105        0             1   
4  2570268               96        301        0             0   

   CC_ALZHEIMER_PLUS  CC_AMI  CC_ANEMIA  CC_ANXIETY_DISORDER  CC_ARTHRITIS  \
0                  1       0          1                    0             1   
1                  1       0          1                    0             1   
2                  1       0          1                    0             1   
3                  1       0          1                    0             1   
4                  1       0          1                    0             1   

   ...  CC_SCHIZOPHRENIA_OTHER  CC_SPINAL_CORD  CC_SPINA_BIFIDA  CC_STROKE  \
0  ...                       0               0                0          0   
1  ...                       0               0                0          0   
2  ...                       0               0                0          0   
3  ...                       0               0                0          0   
4  ...                       0               0                0          0   

   CC_TOBACCO  CC_TRAUMATIC_BRAIN  CC_VISUAL_IMPAIR  MBR_GNDR     MYR  \
0           0                   0                 0         F  201811   
1           0                   0                 0         F  201911   
2           0                   0                 0         F  202005   
3           0                   0                 0         F  201902   
4           0                   0                 0         F  202004   

     MYR_DATE  
0  2018-11-01  
1  2019-11-01  
2  2020-05-01  
3  2019-02-01  
4  2020-04-01  

[5 rows x 75 columns]

### Drop Unesessary Columns from all datasets to reduce computation time

In [19]:
canary_keep = ['PERS_ID', 'REGISTER_DATE', 'LAST_ACTIVITY_DATE']
canary2 = canaryf[canary_keep]

biometrics_keep = ['PERS_ID', 'EVENT_DATE2', 'BIOMETRIC_CATEGORY', 'RESULT_VALUE']
bio2 = bio_catf[biometrics_keep]

#### Standarize dates in the Year-Month Level (making all day values  1)

In [20]:
def standarize_dates( row ):
    year = row.year
    month = row.month
    day = 1
    
    new_row = dt.date(year, month, day)
    
    return new_row

def datetime_to_MYR (row):
    year = row.year*100
    month = row.month
    
    return int(year+month)

#### Standarize Dates
1. <b>Canary (Resigister and Last Activity Dates): it was (Year/ Month/ Day) --> Year/ Month/ Day = 1
2. <b>Biometrics (event Dates): it was (Year/ Month/ Day/ time) --> Year/ Month/ Day = 1

In [21]:
%%time
canary2['REGISTER_DATE'] = canary2.apply(lambda x: standarize_dates(x['REGISTER_DATE']), axis = 1)
canary2['LAST_ACTIVITY_DATE'] = canary2.apply(lambda x: standarize_dates(x['LAST_ACTIVITY_DATE']), axis = 1)
canary2['REGISTER_MYR'] = canary2.apply(lambda x: datetime_to_MYR(x['REGISTER_DATE']), axis = 1)
canary2['LAST_ACTIVITY_MYR'] = canary2.apply(lambda x: datetime_to_MYR(x['LAST_ACTIVITY_DATE']), axis = 1)

bio2['EVENT_DATE2'] = bio2['EVENT_DATE2'].apply( standarize_dates )
bio2['MYR']  = bio2['EVENT_DATE2'].apply( datetime_to_MYR )

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

CPU times: user 4.98 s, sys: 317 ms, total: 5.3 s
Wall time: 5.3 s


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


#### Average Values in the same Month in Biometrics (we had several values from BMi measurements in the same day so we average them out)

In [22]:
%%time
#AVERAGE BIOMETRICS in same month
bio3 = bio2.groupby(by = ['PERS_ID', 'MYR'])['RESULT_VALUE'].mean()

CPU times: user 579 ms, sys: 199 ms, total: 778 ms
Wall time: 777 ms


In [23]:
bio2.drop_duplicates(subset = ['PERS_ID', 'MYR'], inplace = True)
bio2.reset_index(inplace = True, drop = True)

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [24]:
bio3 = bio3.reset_index()

In [25]:
bio3.head()

PERS_ID     MYR  RESULT_VALUE
0  2562798  201106     21.614485
1  2562798  201107     21.440001
2  2562798  201110     21.260000
3  2562798  201201     20.850000
4  2562798  201204     20.730000

#### Create the new biometrics Dataset with average values and sorted indexes

In [26]:
%%time
bio4 = pd.merge(bio3, bio2, left_index = True, right_index = True)
bio4.sort_index(axis = 0, level = 0, inplace=True)
bio4.reset_index(inplace = True)
bio4.drop(['RESULT_VALUE_y', 'PERS_ID_y', 'MYR_y'], axis = 1, inplace = True)
bio4.rename(columns = {'RESULT_VALUE_x': 'RESULT_VALUE', 'PERS_ID_x':'PERS_ID', 'MYR_x':'MYR'}, inplace = True)

CPU times: user 254 ms, sys: 148 ms, total: 402 ms
Wall time: 402 ms


In [27]:
bio4.head()

index  PERS_ID     MYR  RESULT_VALUE EVENT_DATE2 BIOMETRIC_CATEGORY
0      0  2562798  201106     21.614485  2019-10-01                BMI
1      1  2562798  201107     21.440001  2018-12-01                BMI
2      2  2562798  201110     21.260000  2005-11-01                BMI
3      3  2562798  201201     20.850000  2019-12-01                BMI
4      4  2562798  201204     20.730000  2018-08-01                BMI

#### Drop all ids that do not have at least two Values (because we cannot Measure The reference)

In [28]:
values_per_id = bio4.groupby(by = 'PERS_ID')['RESULT_VALUE'].apply(len)

In [29]:
to_drop = values_per_id > 1
values_per_id2 = values_per_id[to_drop]
values_per_id2.name = 'lengths'
print(len(values_per_id2))

102584


In [30]:
bio5 = bio4.merge(values_per_id2, left_on = 'PERS_ID',right_index = True)

In [31]:
del bio_cat, bio2, bio3, biometrics

#### Drop Redundant ids from pmpm and canary

In [32]:
bio5_ids = bio5.PERS_ID.unique()
pmpmf = pmpmf[pmpmf.PERS_ID.isin(bio5_ids)]
canary3 = canary2[canary2.PERS_ID.isin(bio5_ids)]

In [33]:
pmpmf.shape, bio5.shape

((2361215, 75), (1478564, 7))

#### CREATE MERGED PMPM - BIOMETRICS FOR SURVIVAL ANALYSIS (MERGE BY ID AND DATE)

In [34]:
pmpm_bio = pd.merge(bio5, pmpmf, on = ['PERS_ID', 'MYR'])

In [35]:
pmpm_bio_lengths = pmpm_bio.groupby(by = 'PERS_ID').apply(len)

In [36]:
pmpm_bio_lengths2 = pmpm_bio_lengths[pmpm_bio_lengths > 1]

In [37]:

pmpm_bio = pmpm_bio[pmpm_bio.PERS_ID.isin(pmpm_bio_lengths2.index)]

#### Get the common ids for Canary and include a Treated Variable for treatead ids

In [38]:
canary3  = canary2[canary2.PERS_ID.isin(pmpm_bio.PERS_ID.unique())]
canary_ids = canary3.PERS_ID.unique()
pmpm_bio['TREATED'] = 0
pmpm_bio.loc[pmpm_bio.PERS_ID.isin(canary_ids),'TREATED'] = 1

#### Stable sort by Ids and Dates

In [39]:
pmpm_bio.sort_values(by = ['PERS_ID', 'MYR'], inplace = True)

#### Separate Controls and Treated subjects

In [40]:
treated = pmpm_bio[pmpm_bio.TREATED == 1]
control = pmpm_bio[pmpm_bio.TREATED != 1]

In [42]:
len(treated), len(control), len(pmpm_bio)

(6893, 463357, 470250)

#### For Treated We know Register and End Date so we can merge the canary data with the Treated data

In [43]:
treated = treated.merge(canary3, on = 'PERS_ID')

#### For the Control we Need to Find some Artificial Start and End Date (Min - Max)

In [44]:
control.head()

index  PERS_ID     MYR  RESULT_VALUE EVENT_DATE2 BIOMETRIC_CATEGORY  \
0     32  2562798  201805     24.930000  2020-05-01                BMI   
1     33  2562798  201806     24.480000  2019-07-01                BMI   
2     34  2562798  201807     24.120001  2020-03-01                BMI   
3     35  2562798  201809     24.838528  2019-09-01                BMI   
4     36  2562798  201810     25.560000  2020-02-01                BMI   

   lengths  AGE_AT_MIDMONTH  ALLOW_AMT  CC_ADHD  ...  CC_SCHIZOPHRENIA_OTHER  \
0       49              104        377        0  ...                       0   
1       49              104        362        0  ...                       0   
2       49              104        842        0  ...                       0   
3       49              104        313        0  ...                       0   
4       49              104        359        0  ...                       0   

   CC_SPINAL_CORD  CC_SPINA_BIFIDA  CC_STROKE  CC_TOBACCO  CC_TRAUMATIC_BRAIN  \
0               0                0          0           0                   0   
1               0                0          0           0                   0   
2               0                0          0           0                   0   
3               0                0          0           0                   0   
4               0                0          0           0                   0   

   CC_VISUAL_IMPAIR  MBR_GNDR    MYR_DATE  TREATED  
0                 0         F  2018-05-01        0  
1                 0         F  2018-06-01        0  
2                 0         F  2018-07-01        0  
3                 0         F  2018-09-01        0  
4                 0         F  2018-10-01        0  

[5 rows x 81 columns]

In [45]:
def reference_dates( group ):
    """
    Assumption:
    each group is sorted by date
    
    """
    
    length  = len(group)
    myr     = group['MYR']
    
    return pd.DataFrame([[group.PERS_ID.iloc[0], myr.iloc[0], myr.iloc[-1]]], columns = ['PERS_ID','REGISTER_MYR', 'LAST_ACTIVITY_MYR'])
    
    

In [46]:
%%time
control_reference = control.groupby(by = 'PERS_ID').apply(reference_dates)
control_reference.index = range(len(control_reference))

CPU times: user 1min 5s, sys: 317 ms, total: 1min 5s
Wall time: 1min 5s


In [47]:
%%time
control = control.merge(control_reference, on = 'PERS_ID')

CPU times: user 395 ms, sys: 457 ms, total: 852 ms
Wall time: 851 ms


In [48]:
control.head()

index  PERS_ID     MYR  RESULT_VALUE EVENT_DATE2 BIOMETRIC_CATEGORY  \
0     32  2562798  201805     24.930000  2020-05-01                BMI   
1     33  2562798  201806     24.480000  2019-07-01                BMI   
2     34  2562798  201807     24.120001  2020-03-01                BMI   
3     35  2562798  201809     24.838528  2019-09-01                BMI   
4     36  2562798  201810     25.560000  2020-02-01                BMI   

   lengths  AGE_AT_MIDMONTH  ALLOW_AMT  CC_ADHD  ...  CC_SPINA_BIFIDA  \
0       49              104        377        0  ...                0   
1       49              104        362        0  ...                0   
2       49              104        842        0  ...                0   
3       49              104        313        0  ...                0   
4       49              104        359        0  ...                0   

   CC_STROKE  CC_TOBACCO  CC_TRAUMATIC_BRAIN  CC_VISUAL_IMPAIR  MBR_GNDR  \
0          0           0                   0                 0         F   
1          0           0                   0                 0         F   
2          0           0                   0                 0         F   
3          0           0                   0                 0         F   
4          0           0                   0                 0         F   

     MYR_DATE  TREATED  REGISTER_MYR  LAST_ACTIVITY_MYR  
0  2018-05-01        0        201805             201909  
1  2018-06-01        0        201805             201909  
2  2018-07-01        0        201805             201909  
3  2018-09-01        0        201805             201909  
4  2018-10-01        0        201805             201909  

[5 rows x 83 columns]

#### The Last Step is to Separate the Covariates (start and End Covariates for the Matching)
For each row we have the MYR and we know the Beginning and End of the Study, so we choose covariates as close as possible to these dates, in order to calculate the outcome

In [49]:
treated.head()

index  PERS_ID     MYR  RESULT_VALUE EVENT_DATE2 BIOMETRIC_CATEGORY  \
0  72256  2602482  201803     29.094867  2019-07-01                BMI   
1  72257  2602482  201807     27.786186  2020-01-01                BMI   
2  72258  2602482  201808     27.800000  2016-11-01                BMI   
3  72259  2602482  201904     27.583445  2014-11-01                BMI   
4  72260  2602482  201907     27.772373  2017-05-01                BMI   

   lengths  AGE_AT_MIDMONTH  ALLOW_AMT  CC_ADHD  ...  CC_TOBACCO  \
0        9               80       1232        0  ...           0   
1        9               80       2874        0  ...           0   
2        9               80        244        0  ...           0   
3        9               81        244        0  ...           0   
4        9               81        414        0  ...           0   

   CC_TRAUMATIC_BRAIN  CC_VISUAL_IMPAIR  MBR_GNDR    MYR_DATE  TREATED  \
0                   0                 0         F  2018-03-01        1   
1                   0                 0         F  2018-07-01        1   
2                   0                 0         F  2018-08-01        1   
3                   0                 0         F  2019-04-01        1   
4                   0                 0         F  2019-07-01        1   

   REGISTER_DATE  LAST_ACTIVITY_DATE  REGISTER_MYR  LAST_ACTIVITY_MYR  
0     2019-08-01          2019-08-01        201908             201908  
1     2019-08-01          2019-08-01        201908             201908  
2     2019-08-01          2019-08-01        201908             201908  
3     2019-08-01          2019-08-01        201908             201908  
4     2019-08-01          2019-08-01        201908             201908  

[5 rows x 85 columns]

#### Calculate Distance From REGISTARTION AND LAST_ACTIVITY DATE TO EACH MYR

In [50]:
treated['REF_START']  = np.abs(treated.MYR_DATE - treated.REGISTER_DATE)
treated['REF_END']  = np.abs(treated.MYR_DATE - treated.LAST_ACTIVITY_DATE)

In [51]:
def pick_covariates( group ):
    """
    choses rows closer 
    to registration and last activity date
    
    """
    indx_start = np.argmin(group['REF_START'].values)
    indx_end   = np.argmin(group['REF_END'].values)
    N = len(group) - 1
    if indx_start == indx_end:
        if indx_start != N:
            indx_end = N
        else:
            indx_start = 0
            
    data = group.iloc[[indx_start, indx_end]]    
    data['FF'] = 0
    data.iloc[1,-1] = 1
    data['OUTCOME'] = int((data['RESULT_VALUE'].iloc[0] - data['RESULT_VALUE'].iloc[-1])> 0)
    
    return data
        
        
    

In [52]:
%%time
treated2 = treated.groupby(by = 'PERS_ID').apply(pick_covariates)
treated2.index = range(len(treated2))

CPU times: user 3.08 s, sys: 0 ns, total: 3.08 s
Wall time: 3.08 s


In [53]:
treated2.head()

index  PERS_ID     MYR  RESULT_VALUE EVENT_DATE2 BIOMETRIC_CATEGORY  \
0   72260  2602482  201907     27.772373  2017-05-01                BMI   
1   72264  2602482  202003     27.201864  2017-01-01                BMI   
2   73560  2603072  202001     29.256837  2015-01-01                BMI   
3   73561  2603072  202008     29.467907  2018-06-01                BMI   
4  116199  2621187  202001     23.289228  2019-01-01                BMI   

   lengths  AGE_AT_MIDMONTH  ALLOW_AMT  CC_ADHD  ...    MYR_DATE  TREATED  \
0        9               81        414        0  ...  2019-07-01        1   
1        9               82        351        0  ...  2020-03-01        1   
2       31               82        101        0  ...  2020-01-01        1   
3       31               82        198        0  ...  2020-08-01        1   
4       10               78      13007        0  ...  2020-01-01        1   

   REGISTER_DATE  LAST_ACTIVITY_DATE  REGISTER_MYR  LAST_ACTIVITY_MYR  \
0     2019-08-01          2019-08-01        201908             201908   
1     2019-08-01          2019-08-01        201908             201908   
2     2020-01-01          2020-06-01        202001             202006   
3     2020-01-01          2020-06-01        202001             202006   
4     2020-01-01          2020-01-01        202001             202001   

   REF_START  REF_END  FF  OUTCOME  
0    31 days  31 days   0        1  
1   213 days 213 days   1        1  
2     0 days 152 days   0        0  
3   213 days  61 days   1        0  
4     0 days   0 days   0        0  

[5 rows x 89 columns]

#### AT THIS POINT WE HAVE ALL THE COVARIATES FOR THE TREATED AND WE CAN USE THEM FOR MATCHING, FOR THE CONTROLS WE CALCULATE THE COVARIATES IN THE MATCHING NOTEBOOK

In [54]:
treated2

index  PERS_ID     MYR  RESULT_VALUE EVENT_DATE2 BIOMETRIC_CATEGORY  \
0       72260  2602482  201907     27.772373  2017-05-01                BMI   
1       72264  2602482  202003     27.201864  2017-01-01                BMI   
2       73560  2603072  202001     29.256837  2015-01-01                BMI   
3       73561  2603072  202008     29.467907  2018-06-01                BMI   
4      116199  2621187  202001     23.289228  2019-01-01                BMI   
...       ...      ...     ...           ...         ...                ...   
1897  1479261  5025690  202008     42.629629  2014-10-01                BMI   
1898  1487480  5036084  202005     35.702410  2020-04-01                BMI   
1899  1487483  5036084  202009     34.328087  2018-10-01                BMI   
1900  1487933  5036817  202005     52.110743  2019-11-01                BMI   
1901  1487934  5036817  202006     51.391768  2016-07-01                BMI   

      lengths  AGE_AT_MIDMONTH  ALLOW_AMT  CC_ADHD  ...    MYR_DATE  TREATED  \
0           9               81        414        0  ...  2019-07-01        1   
1           9               82        351        0  ...  2020-03-01        1   
2          31               82        101        0  ...  2020-01-01        1   
3          31               82        198        0  ...  2020-08-01        1   
4          10               78      13007        0  ...  2020-01-01        1   
...       ...              ...        ...      ...  ...         ...      ...   
1897       10               45       2053        0  ...  2020-08-01        1   
1898       41               49        628        0  ...  2020-05-01        1   
1899       41               50        161        0  ...  2020-09-01        1   
1900       36               45        618        0  ...  2020-05-01        1   
1901       36               45       3293        0  ...  2020-06-01        1   

      REGISTER_DATE  LAST_ACTIVITY_DATE  REGISTER_MYR  LAST_ACTIVITY_MYR  \
0        2019-08-01          2019-08-01        201908             201908   
1        2019-08-01          2019-08-01        201908             201908   
2        2020-01-01          2020-06-01        202001             202006   
3        2020-01-01          2020-06-01        202001             202006   
4        2020-01-01          2020-01-01        202001             202001   
...             ...                 ...           ...                ...   
1897     2020-07-01          2020-08-01        202007             202008   
1898     2020-05-01          2020-09-01        202005             202009   
1899     2020-05-01          2020-09-01        202005             202009   
1900     2020-05-01          2020-08-01        202005             202008   
1901     2020-05-01          2020-08-01        202005             202008   

      REF_START  REF_END  FF  OUTCOME  
0       31 days  31 days   0        1  
1      213 days 213 days   1        1  
2        0 days 152 days   0        0  
3      213 days  61 days   1        0  
4        0 days   0 days   0        0  
...         ...      ...  ..      ...  
1897    31 days   0 days   1        0  
1898     0 days 123 days   0        1  
1899   123 days   0 days   1        1  
1900     0 days  92 days   0        1  
1901    31 days  61 days   1        1  

[1902 rows x 89 columns]

#### SAVE THE TWO DATASETS FOR PROCESSING IN  DIFFERENT NOTEBOOKS

In [55]:
p = Path('../../datap')
p.mkdir(parents = True, exist_ok = True)

In [56]:
control.to_pickle('../../datap/control.p')
treated2.to_pickle('../../datap/treated.p')

In [57]:
control.head()

index  PERS_ID     MYR  RESULT_VALUE EVENT_DATE2 BIOMETRIC_CATEGORY  \
0     32  2562798  201805     24.930000  2020-05-01                BMI   
1     33  2562798  201806     24.480000  2019-07-01                BMI   
2     34  2562798  201807     24.120001  2020-03-01                BMI   
3     35  2562798  201809     24.838528  2019-09-01                BMI   
4     36  2562798  201810     25.560000  2020-02-01                BMI   

   lengths  AGE_AT_MIDMONTH  ALLOW_AMT  CC_ADHD  ...  CC_SPINA_BIFIDA  \
0       49              104        377        0  ...                0   
1       49              104        362        0  ...                0   
2       49              104        842        0  ...                0   
3       49              104        313        0  ...                0   
4       49              104        359        0  ...                0   

   CC_STROKE  CC_TOBACCO  CC_TRAUMATIC_BRAIN  CC_VISUAL_IMPAIR  MBR_GNDR  \
0          0           0                   0                 0         F   
1          0           0                   0                 0         F   
2          0           0                   0                 0         F   
3          0           0                   0                 0         F   
4          0           0                   0                 0         F   

     MYR_DATE  TREATED  REGISTER_MYR  LAST_ACTIVITY_MYR  
0  2018-05-01        0        201805             201909  
1  2018-06-01        0        201805             201909  
2  2018-07-01        0        201805             201909  
3  2018-09-01        0        201805             201909  
4  2018-10-01        0        201805             201909  

[5 rows x 83 columns]

In [58]:
treated2.head()

index  PERS_ID     MYR  RESULT_VALUE EVENT_DATE2 BIOMETRIC_CATEGORY  \
0   72260  2602482  201907     27.772373  2017-05-01                BMI   
1   72264  2602482  202003     27.201864  2017-01-01                BMI   
2   73560  2603072  202001     29.256837  2015-01-01                BMI   
3   73561  2603072  202008     29.467907  2018-06-01                BMI   
4  116199  2621187  202001     23.289228  2019-01-01                BMI   

   lengths  AGE_AT_MIDMONTH  ALLOW_AMT  CC_ADHD  ...    MYR_DATE  TREATED  \
0        9               81        414        0  ...  2019-07-01        1   
1        9               82        351        0  ...  2020-03-01        1   
2       31               82        101        0  ...  2020-01-01        1   
3       31               82        198        0  ...  2020-08-01        1   
4       10               78      13007        0  ...  2020-01-01        1   

   REGISTER_DATE  LAST_ACTIVITY_DATE  REGISTER_MYR  LAST_ACTIVITY_MYR  \
0     2019-08-01          2019-08-01        201908             201908   
1     2019-08-01          2019-08-01        201908             201908   
2     2020-01-01          2020-06-01        202001             202006   
3     2020-01-01          2020-06-01        202001             202006   
4     2020-01-01          2020-01-01        202001             202001   

   REF_START  REF_END  FF  OUTCOME  
0    31 days  31 days   0        1  
1   213 days 213 days   1        1  
2     0 days 152 days   0        0  
3   213 days  61 days   1        0  
4     0 days   0 days   0        0  

[5 rows x 89 columns]